In [ ]:
import os
from pathlib import Path
import geopandas as gpd
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

In [ ]:
def fetch_and_save_image(region_id, minx, miny, maxx, maxy, folder_path, base_url):
    file_path = os.path.join(folder_path, f"{region_id}.jpg")
    if os.path.exists(file_path):
        return  # Skip if the image already exists

    bbox_string = f"{minx},{miny},{maxx},{maxy}"
    params = {
        'service': 'WMS',
        'request': 'GetMap',
        'version': '1.3.0',
        'layers': 'Actueel_orthoHR',
        'styles': '',
        'bbox': bbox_string,
        'width': '224',
        'height': '224',
        'crs': 'EPSG:28992',
        'format': 'image/jpeg',
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            file.write(response.content)
    else:
        print(f"Failed to retrieve the image for region {region_id}. Status code:", response.status_code)

def download_missing_images(regions_gdf, images_dir, base_url, max_workers=36):
    os.makedirs(images_dir, exist_ok=True)
    existing_images = set(f.stem for f in Path(images_dir).glob("*.jpg"))
    all_region_ids = set(regions_gdf.index.astype(str))
    missing_region_ids = all_region_ids - existing_images
    missing_regions_gdf = regions_gdf.loc[list(missing_region_ids)]

    print(f"Number of existing images: {len(existing_images)}")
    print(f"Number of missing images: {len(missing_region_ids)}")

    args_list = [(region_id, row['minx'], row['miny'], row['maxx'], row['maxy'], images_dir, base_url)
                 for region_id, row in missing_regions_gdf.iterrows()]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(lambda args: fetch_and_save_image(*args), args_list), total=len(args_list)))

In [ ]:
# Usage
regions_buffered_gdf = gpd.read_file("selected_regions_buffered_10.geojson")
regions_buffered_gdf = regions_buffered_gdf.to_crs(epsg=28992)
regions_buffered_gdf.set_index('region_id', inplace=True)
bbox = regions_buffered_gdf.geometry.bounds
regions_buffered_gdf['minx'] = bbox['minx']
regions_buffered_gdf['miny'] = bbox['miny']
regions_buffered_gdf['maxx'] = bbox['maxx']
regions_buffered_gdf['maxy'] = bbox['maxy']

images_dir = "D://tu delft//Afstuderen//aerial_images_10"
base_url = "https://service.pdok.nl/hwh/luchtfotorgb/wms/v1_0"

download_missing_images(regions_buffered_gdf, images_dir, base_url)